In [7]:
# Preliminaries

# scratch_location = r'/scratch/hmnshpl'
import os
import sys
import heapq
import getpass
import numpy as np
import pandas as pd
import networkx as nx
from copy import deepcopy
from collections import defaultdict

dataset_name = 'wikipedia'
scratch_location = rf'/scratch/{getpass.getuser()}'


## Load Data
# Load data and train val test split
graph_df = pd.read_csv('{}/processed_data/{}/ml_{}.csv'.format(scratch_location,
                                                            dataset_name,
                                                            dataset_name)
                    )
edge_raw_features = np.load('{}/processed_data/{}/ml_{}.npy'.format(scratch_location,
                                                                    dataset_name,
                                                                    dataset_name)
                            )
node_raw_features = np.load('{}/processed_data/{}/ml_{}_node.npy'.format(scratch_location,
                                                                        dataset_name,
                                                                        dataset_name)
                            )

# Set the working directory to the project root
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')) # this might cause issue
sys.path.append(project_root)

### Temperal EdgeRank Implementation

In [71]:
def temporal_pagerank_heap_np(E, beta, alpha, check_evolution=False):
    # print('\t inside tpr heap method')
    # Convert edges to a NumPy array
    E = np.array(E, dtype=[('u', int), ('v', int), ('t', float)])
    
    # Get the maximum node index to size the r and s arrays appropriately
    max_node = max(E['u'].max(), E['v'].max())
    
    # Initialize r and s arrays
    r = np.zeros(max_node + 1)
    s = np.zeros(max_node + 1)
    
    ts_tpr = [] if check_evolution else None
    
    # Use a heap to efficiently process edges in time order
    heap = [(t, u, v) for u, v, t in E]
    heapq.heapify(heap)
    # print('\t heapify successful')
    while heap:
        t, u, v = heapq.heappop(heap)
        
        # Update r and s values
        delta = 1 - alpha
        r[u] += delta
        s[u] += delta
        r[v] += s[u] * alpha
        
        if beta < 1:
            s_v_increment = s[u] * (1 - beta) * alpha
            s[v] += s_v_increment
            s[u] *= beta
        else:
            s[v] += s[u] * alpha
            s[u] = 0
        
        # Store evolution if required
        if check_evolution:
            # ts_tpr.append((t, r.copy()))  # Store r values at current timestamp
            # Normalize r before appending
            total_r = r.sum()
            if total_r > 0:
                ts_tpr.append((t, r.copy() / total_r))
    # print('\t out of loop.')
    
    # Normalize r
    total_r = r.sum()
    if total_r > 0:
        r /= total_r
    
    if check_evolution:
        ts_tpr = np.array(ts_tpr, dtype=[('t', float), ('r', float, max_node + 1)])
    
    return r, ts_tpr

In [6]:
# Extract nodes, edges, and timestamps
edges = graph_df[['u', 'i', 'ts']].values
nodes = np.unique(edges[:, :2])  # Get unique nodes from edges

# Convert E to a more readable format if needed
edges_new = [(int(u), int(v), float(t)) for u, v, t in edges]

beta = 0.85
alpha = 0.15

r2, ts_tpr= temporal_pagerank_heap_np(edges_new, beta, alpha, True)
print(r2)

	 inside tpr heap method
	 heapify successful
	 out of loop.
[0.00000000e+00 3.40154070e-06 9.69439101e-04 ... 2.47259416e-04
 2.89130960e-04 5.06829565e-04]


In [8]:
def compute_temporal_outgoing_degree(E):
    E = np.array(E, dtype=[('u', int), ('v', int), ('t', float)])
    outgoing_degree = defaultdict(int)
    temporal_outgoing_degree = defaultdict(list)
    heap = [(t, u, v) for u, v, t in E]
    heapq.heapify(heap)
    while heap:
        t, u, v = heapq.heappop(heap)
        outgoing_degree[u] += 1
        for node in outgoing_degree:
            temporal_outgoing_degree[node].append((t, outgoing_degree[node]))
    return temporal_outgoing_degree

In [52]:
from tqdm import tqdm


def compute_temporal_edgerank(E, beta, alpha):
    _, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
    print(len(ts_tpr))
    temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
    temporal_edgerank = defaultdict(lambda: defaultdict(list))
    
    for tpr in tqdm(ts_tpr, desc='Calculating Temporal EdgeRank'):
        t, r = tpr['t'], tpr['r']
        for u, v, t_ev in E:
            if t_ev <= t:
                if u in r and u in temporal_outgoing_degree:
                    current_degree = [d for time, d in temporal_outgoing_degree[u] if time <= t][-1]
                    if current_degree > 0:
                        edge_rank = r[u] / current_degree
                    else:
                        edge_rank = 0
                    print('Add rank...')
                    temporal_edgerank[u][v].append((t, edge_rank))
    
    return temporal_edgerank

In [20]:
# Example usage:
E = [
    (1, 2, 1.0),
    (2, 3, 2.0),
    (1, 3, 3.0),
    (1, 2, 4.0),
    (2, 1, 5.0)
]

temporal_edgerank = compute_temporal_edgerank(E, beta, alpha)

# Print Temporal EdgeRank
for u in temporal_edgerank:
    for v in temporal_edgerank[u]:
        print(f"Edge ({u}, {v}) Temporal EdgeRank over time: {temporal_edgerank[u][v]}")

	 inside tpr heap method
	 heapify successful
	 out of loop.
5


Calculating Temporal EdgeRank: 100%|██████████| 5/5 [00:00<00:00, 6180.82it/s]


In [22]:
_, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
ts_tpr

	 inside tpr heap method
	 heapify successful
	 out of loop.


array([(1., [0.        , 0.86956522, 0.13043478, 0.        ]),
       (2., [0.        , 0.43414555, 0.49926738, 0.06658707]),
       (3., [0.        , 0.55852271, 0.32115056, 0.12032674]),
       (4., [0.        , 0.60401671, 0.30923139, 0.0867519 ]),
       (5., [0.        , 0.52576017, 0.40536376, 0.06887607])],
      dtype=[('t', '<f8'), ('r', '<f8', (4,))])

In [23]:
temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
temporal_outgoing_degree

defaultdict(list,
            {1: [(1.0, 1), (2.0, 1), (3.0, 2), (4.0, 3), (5.0, 3)],
             2: [(2.0, 1), (3.0, 1), (4.0, 1), (5.0, 2)]})

In [38]:
temporal_edgerank = defaultdict(lambda: defaultdict(list))

for tpr in tqdm(ts_tpr, desc='Calculating Temporal EdgeRank'):
    t, r = tpr['t'], tpr['r']
    for u, v, t_ev in E:
        if t_ev <= t:
            if u in temporal_outgoing_degree:
                current_degree = [d for time, d in temporal_outgoing_degree[u] if time <= t][-1]
                if current_degree > 0:
                    edge_rank = r[u] / current_degree
                else:
                    edge_rank = 0
                print('Add rank...')
                temporal_edgerank[u][v].append((t, edge_rank))
                
len(temporal_edgerank)

Calculating Temporal EdgeRank: 100%|██████████| 5/5 [00:00<00:00, 11580.08it/s]

Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...
Add rank...


2

In [36]:
for u, v, t_ev in E:
    r = dict(ts_tpr)[t_ev]
    if u in temporal_outgoing_degree:
        current_degree = [d for time, d in temporal_outgoing_degree[u] if time <= t][-1]
        if current_degree > 0:
            edge_rank = r[u] / current_degree
        else:
            edge_rank = 0
        print('Add rank...')
        temporal_edgerank[u][v].append((t, edge_rank))
len(temporal_edgerank)        

Add rank...
Add rank...
Add rank...
Add rank...
Add rank...


2

In [37]:
temporal_edgerank

defaultdict(<function __main__.<lambda>()>,
            {1: defaultdict(list,
                         {2: [(5.0, 0.2898550724637681),
                           (5.0, 0.2013389037096693)],
                          3: [(5.0, 0.18617423581294143)]}),
             2: defaultdict(list,
                         {3: [(5.0, 0.24963368969447008)],
                          1: [(5.0, 0.20268188081232408)]})})

In [39]:
temporal_edgerank

defaultdict(<function __main__.<lambda>()>,
            {1: defaultdict(list,
                         {2: [(1.0, 0.8695652173913043),
                           (2.0, 0.43414554729473054),
                           (3.0, 0.27926135371941213),
                           (4.0, 0.2013389037096693),
                           (4.0, 0.2013389037096693),
                           (5.0, 0.17525338856740116),
                           (5.0, 0.17525338856740116)],
                          3: [(3.0, 0.27926135371941213),
                           (4.0, 0.2013389037096693),
                           (5.0, 0.17525338856740116)]}),
             2: defaultdict(list,
                         {3: [(2.0, 0.49926737938894017),
                           (3.0, 0.321150556777324),
                           (4.0, 0.3092313887350884),
                           (5.0, 0.20268188081232408)],
                          1: [(5.0, 0.20268188081232408)]})})

In [94]:
def compute_temporal_edgerank_with_time_decay_smoothening(E, beta, alpha, decay_factor=0.9):
    _, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
    temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
    assert len(ts_tpr) > 0, 'Check1'
    assert len(temporal_outgoing_degree) > 0, 'Check2'
    temporal_edgerank = defaultdict(lambda: defaultdict(list))
    
    def smooth_degree(degree):
        return degree ** 0.5  # Smoothing function (e.g., square root)
    
    for tpr in ts_tpr:
        t, r = tpr['t'], tpr['r']
        print(f'for {t=}')
        for u, v, t_ev in E:
            print(f'\tProcessing {t_ev=} for {(u, v)}', end=' ')
            if t_ev <= t:
                print('started: ')
                current_degrees = [d for time, d in temporal_outgoing_degree[u] if time <= t]
                if current_degrees:
                    current_degree = smooth_degree(current_degrees[-1])
                    if current_degree > 0:
                        time_diff = t - t_ev
                        edge_rank = (r[u] / current_degree) * (decay_factor ** time_diff)
                    else:
                        edge_rank = 0
                    temporal_edgerank[u][v].append((t, edge_rank))
                    print(f'\t\tappended {(u, v)} for time {t}')
            else:
                print('Unprocessed.', end='\n')
    assert len(temporal_edgerank) != 0, 'Check2'
    
    return temporal_edgerank

In [136]:
def compute_temporal_edgerank_with_time_decay_smoothening(E, beta, alpha, decay_factor=0.9):
    _, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
    temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
    assert len(ts_tpr) > 0, 'Check1'
    assert len(temporal_outgoing_degree) > 0, 'Check2'
    temporal_edgerank = defaultdict(lambda: defaultdict(list))

    def smooth_degree(degree):
        return degree ** 0.5  # Smoothing function (e.g., square root)
    
    for tpr in ts_tpr:
        t, r = tpr['t'], tpr['r']
        print(f'Processing timestamp {t}')
        
        processed_edges = set()  # Track processed (u, v, t_ev) tuples
        
        for u, v, t_ev in E:
            print(f'\t{(u, v, t_ev)}', end = ' ')
            if (u, v, t_ev) in processed_edges:
                print(f' already processed. Skipped...')
                continue  # Skip if already processed
            # print('')
            if t_ev <= t: # changed here from t_ev <= t to t_ev == t
                print(f'\tProcessing edge ({u}, {v}) at time {t_ev} for timestamp {t}')
                
                # Retrieve degrees up to current timestamp t
                current_degrees = [d for time, d in temporal_outgoing_degree[u] if time <= t]  # changed here from time <= t to time == t
                if current_degrees:
                    current_degree = smooth_degree(current_degrees[-1])
                    if current_degree > 0:
                        time_diff = t - t_ev
                        edge_rank = (r[u] / current_degree) * (decay_factor ** time_diff)
                    else:
                        edge_rank = 0
                    temporal_edgerank[u][v].append((t, edge_rank))
                    print(f'\t\t\t\t\tAppended edge rank: ({u}, {v}, {t}, {edge_rank})')
                else:
                    print(f'\t\tNo valid degree for user {u} at time {t}')
                
                processed_edges.add((u, v, t_ev))  # Mark this edge as processed
            else:
                print(f'\tSkipping edge ({u}, {v}) at time {t_ev} as it is not != {t}')
    
    assert len(temporal_edgerank) != 0, 'Check2'
    return temporal_edgerank

In [140]:
def compute_temporal_edgerank_with_time_decay_smoothening(E, beta, alpha, decay_factor=0.9):
    _, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
    temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
    assert len(ts_tpr) > 0, 'Check1'
    assert len(temporal_outgoing_degree) > 0, 'Check2'
    temporal_edgerank = defaultdict(lambda: defaultdict(list))
    
    def smooth_degree(degree):
        return degree ** 0.5  # Smoothing function (e.g., square root)
    
    # Create a mapping from edges to their timestamps
    edge_to_time = defaultdict(list)
    for u, v, t_ev in E:
        edge_to_time[(u, v)].append(t_ev)
    
    for tpr in ts_tpr:
        t, r = tpr['t'], tpr['r']
        print(f'Processing timestamp {t}')
        
        for (u, v), times in edge_to_time.items():
            # Process each edge only if it has not been processed for this timestamp
            for t_ev in times:
                if t_ev <= t:
                    print(f'\tProcessing edge ({u}, {v}) at time {t_ev} for timestamp {t}')
                    
                    # Retrieve degrees up to current timestamp t
                    current_degrees = [d for time, d in temporal_outgoing_degree[u] if time <= t]
                    if current_degrees:
                        current_degree = smooth_degree(current_degrees[-1])
                        if current_degree > 0:
                            time_diff = t - t_ev
                            edge_rank = (r[u] / current_degree) * (decay_factor ** time_diff)
                        else:
                            edge_rank = 0
                        temporal_edgerank[u][v].append((t, edge_rank))
                        print(f'\t\t\t\t\tAppended edge rank: ({u}, {v}, {t}, {edge_rank})')
                    
                    # Stop processing this edge after the first valid timestamp
                    break
                else:
                    print(f'\tSkipping edge ({u}, {v}) at time {t_ev} as it is not <= {t}')
    
    assert len(temporal_edgerank) != 0, 'Check2'
    return temporal_edgerank


In [141]:
temporal_edgerank = compute_temporal_edgerank_with_time_decay_smoothening(E, beta, alpha, decay_factor=0.9)
# (temporal_edgerank)

Processing timestamp 1.0
	Processing edge (1, 2) at time 1.0 for timestamp 1.0
					Appended edge rank: (1, 2, 1.0, 0.8695652173913043)
	Skipping edge (2, 3) at time 2.0 as it is not <= 1.0
	Skipping edge (1, 3) at time 3.0 as it is not <= 1.0
	Skipping edge (2, 1) at time 5.0 as it is not <= 1.0
Processing timestamp 2.0
	Processing edge (1, 2) at time 1.0 for timestamp 2.0
					Appended edge rank: (1, 2, 2.0, 0.3907309925652575)
	Processing edge (2, 3) at time 2.0 for timestamp 2.0
					Appended edge rank: (2, 3, 2.0, 0.49926737938894017)
	Skipping edge (1, 3) at time 3.0 as it is not <= 2.0
	Skipping edge (2, 1) at time 5.0 as it is not <= 2.0
Processing timestamp 3.0
	Processing edge (1, 2) at time 1.0 for timestamp 3.0
					Appended edge rank: (1, 2, 3.0, 0.31989750704009684)
	Processing edge (2, 3) at time 2.0 for timestamp 3.0
					Appended edge rank: (2, 3, 3.0, 0.28903550109959164)
	Processing edge (1, 3) at time 3.0 for timestamp 3.0
					Appended edge rank: (1, 3, 3.0, 0.3949

In [130]:
E

[(1, 2, 1.0), (2, 3, 2.0), (1, 3, 3.0), (1, 2, 4.0), (2, 1, 5.0)]

In [131]:
# Function to sort the inner defaultdict
def sort_inner_dict(d):
    return {k: sorted(v, key=lambda x: x[0]) for k, v in d.items()}

# Sort the inner default dicts
sorted_data = {k: sort_inner_dict(v) for k, v in temporal_edgerank.items()}

# Sort the outer defaultdict
sorted_data = dict(sorted(sorted_data.items()))

print(sorted_data)


{1: {2: [(1.0, 0.8695652173913043), (4.0, 0.34872921076536517)], 3: [(3.0, 0.39493519387666276)]}, 2: {3: [(2.0, 0.49926737938894017)], 1: [(5.0, 0.28663546469207585)]}}


In [132]:
# Flatten the defaultdict into a list of records [u, i, ts, value]
records = []
for u, inner_dict in temporal_edgerank.items():
    for i, values in inner_dict.items():
        for ts, value in values:
            records.append([u, i, ts, value])

# Create a DataFrame from the records
df = pd.DataFrame(records, columns=['u', 'i', 'ts', 'value'])

print(df)

   u  i   ts     value
0  1  2  1.0  0.869565
1  1  2  4.0  0.348729
2  1  3  3.0  0.394935
3  2  3  2.0  0.499267
4  2  1  5.0  0.286635


In [133]:
df2 = pd.DataFrame(E)
# Rename the columns in df2 to match those in df1
df2.columns = ['u', 'i', 'ts']

# Merge the DataFrames on 'u', 'i', and 'ts'
merged_df = pd.merge(df2, df, on=['u', 'i', 'ts'])

(merged_df)

,u,i,ts,value
0,1,2,1.0,0.869565
1,2,3,2.0,0.499267
2,1,3,3.0,0.394935
3,1,2,4.0,0.348729
4,2,1,5.0,0.286635


In [68]:
len(df2), len(merged_df)

(5, 6)

In [69]:
df2

,u,i,ts
0,1,2,1.0
1,2,3,2.0
2,1,3,3.0
3,1,2,4.0
4,2,1,5.0
